In [3]:
# Import tools

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:

# Load the data and preprocess as needed
data = np.genfromtxt('magic04.data', delimiter=',', dtype=None, encoding=None)
X = data[:, :-1].astype(np.float32)
y = (data[:, -1] == b'g').astype(np.int32)